In [1]:
import os
import sys
import subprocess
from IPython.display import display, Markdown
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('CLAUDE_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:6]}")
else:
    print("OpenRouter API Key not set (and this is optional)")



OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI
OpenRouter API Key exists and begins sk-or-


In [4]:
openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
ollama_url = "http://localhost:11434/v1"
openrouter_url = "https://openrouter.ai/api/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)
openrouter = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)

In [5]:
models = ["gpt-5", "claude-sonnet-4-5-20250929", "gemini-2.5-pro", "qwen2.5-coder", "deepseek-coder-v2", "gpt-oss:20b", "qwen/qwen3-coder-30b-a3b-instruct"]

clients = {"gpt-5": openai, "claude-sonnet-4-5-20250929": anthropic, "gemini-2.5-pro": gemini, "qwen2.5-coder": ollama, "deepseek-coder-v2": ollama, "gpt-oss:20b": ollama, "qwen/qwen3-coder-30b-a3b-instruct": openrouter}

# Want to keep costs ultra-low? Replace this with models of your choice, using the examples from yesterday

In [6]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

{'os': {'system': 'Darwin',
  'arch': 'arm64',
  'release': '23.4.0',
  'version': 'Darwin Kernel Version 23.4.0: Fri Mar 15 00:19:22 PDT 2024; root:xnu-10063.101.17~1/RELEASE_ARM64_T8112',
  'kernel': '23.4.0',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'arm64-apple-darwin23.4.0'},
 'package_managers': ['xcode-select (CLT)', 'brew'],
 'cpu': {'brand': 'Apple M2',
  'cores_logical': 8,
  'cores_physical': 8,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'Apple clang version 15.0.0 (clang-1500.3.9.4)',
   'g++': 'Apple clang version 15.0.0 (clang-1500.3.9.4)',
   'clang': 'Apple clang version 15.0.0 (clang-1500.3.9.4)',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': 'GNU Make 3.81'},
  'linkers': {'ld_lld': ''}}}

In [8]:
compile_command = ["clang++", "-std=c++20", "-O3", "-DNDEBUG", "-o", "main", "main.cpp"]

run_command = ["./main"]

In [9]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

In [10]:
def user_prompt_for(python_code: str) -> str:
    return f"""
    Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
    The system information is:
    {system_info}
    Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
    {compile_command}
    
    Respond only with C++ code.
    Python code to port:

    ```python
    {python_code}
    ```
    """

In [11]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [12]:
def write_output(cpp):
    with open("main.cpp", "w", encoding="utf-8") as f:
        f.write(cpp)

In [19]:
def port(model, python):
    client = clients[model]
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)

In [20]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [21]:
def run_python(code):
    globals = {"__builtins__": __builtins__}
    exec(code, globals)

In [22]:
run_python(pi)

Result: 3.141592656089
Execution Time: 14.320100 seconds


In [23]:
def compile_and_run():
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred:\n{e.stderr}")

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=28, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=28)
    with gr.Row():
        model = gr.Dropdown(models, label="Select model", value=models[0])
        convert = gr.Button("Convert code")

    convert.click(port, inputs=[model, python], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/rahultangsali/Desktop/llm-engineering-course/llm_engineering/.venv/lib/python3.11/site-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rahultangsali/Desktop/llm-engineering-course/llm_engineering/.venv/lib/python3.11/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rahultangsali/Desktop/llm-engineering-course/llm_engineering/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rahultangsali/Desktop/llm-engineering-course/llm_engineering/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(